In [1]:
import os
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models

C:\Users\Sabrine\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Sabrine\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
filename = os.path.join('hillary-clinton-emails','Emails.csv') 
df = pd.read_csv(filename)
df.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


We deceided first to use only the ExtractedBodyText even if the rawText could have gave us more information.

In [4]:
extractedBodyText = df['ExtractedBodyText']
extractedBodyText.head()

0                                                  NaN
1    B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...
2                                                  Thx
3                                                  NaN
4    H <hrod17@clintonemail.com>\nFriday, March 11,...
Name: ExtractedBodyText, dtype: object

In [5]:
# Dealing with Nan values
extractedBodyText.fillna("", inplace = True)

In [6]:
# Creation of the string we will use for the worldcloud
theText = ""
for i in extractedBodyText.index:
    theText = theText + "".join(extractedBodyText[i])

In [7]:
from wordcloud import WordCloud

In [8]:
wordcloud = WordCloud().generate(theText)

In [9]:
# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(theText)
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

The result is not good at all, words displayed do not exist.
We need to clean the data from this noise.

In [10]:
import re

We remove everything between <>

In [11]:
extractedBodyText2 = extractedBodyText.apply(lambda x: re.sub('<.*>', ' ', x ))

The result is still really bad. We noticed that avery line after 030311.docx were not usefull and could be dropped.

In [13]:
extractedBodyText3 = extractedBodyText2.apply(lambda x: re.sub('030311 .* \\n', '', x))

In [15]:
extractedBodyText4 = extractedBodyText3.apply(lambda x: re.sub('^STATE DEPT .* STATE-[0-9A-Z]+', ' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('Date: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('Doc No.: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('Case No.: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('Sent: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('To: .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('^[A-Z] .* @ .* \\n',' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub('[A-Z]{1}[a-z]+, [A-Z]{1}[a-z]+ \d+, [0-9]+ \d+:\d+ [A-Z]{2}', ' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub(' [a-zA-Z]{1} ', ' ', x))
extractedBodyText4 = extractedBodyText4.apply(lambda x: re.sub(' [a-zA-Z]{2} ', ' ', x))

In [17]:
del extractedBodyText, extractedBodyText2, extractedBodyText3

In [18]:
fourthAttempt = ""
for i in extractedBodyText4.index:
    fourthAttempt = fourthAttempt + "".join(extractedBodyText4[i])

In [19]:
fourthAttempt[:1000]

"B6\n \nH: Latest How Syria aiding Qaddafi and more... Sid\nhrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx\nMarch 3, 2011\nFor: HillaryThxH  \n \nHuma Abedin\nFw: H: Latest: How Syria aiding Qaddafi and more... Sid\nhrc memo syria aiding libya 030311.docx\nPis print.Pis print.\n-•-...-^\nH  \n \n°Russorv@state.gov'\nFw: Meet The Right-Wing Extremist Behind Anti-fvluslim Film That Sparked Deadly Riots\nFrom [meat)\nSent:  \nTo: 11\nSubject: Meet The Right Wing Extremist Behind Anti-Muslim Film That Sparked Deadly Riots\nhtte/maxbiumenthal.com12012/09/meet-the-right-wing-extremist-behind-anti-musiim-tihn-that-sparked-\ndeadly-riots/\nSent from Verizon Wireless 4G LTE DROID\nU.S. Department State\nCase No. F-2015-04841\nDoc No. C05739559\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED HOUSE SELECT BENGHAZI COMM.\nSUBJECT AGREEMENT SENSITIVE INFORMATION & REDACTIONS. FOIA WAIVER. STATE-5CB0045251H  \n \nHuma Abedin\nFw: H: Latest: How Syria aiding Qaddafi and mo

We need to remove the new line symbole "\n"

In [21]:
extractedBodyText5 = extractedBodyText4.apply(lambda x: re.sub('\n', ' ', x))
del extractedBodyText4

In [22]:
fifthAttempt = ""
for i in extractedBodyText5.index:
    fifthAttempt = fifthAttempt + "".join(extractedBodyText5[i])

In [23]:
fifthAttempt[:1000]

"B6   H: Latest How Syria aiding Qaddafi and more... Sid hrc memo syria aiding libya 030311.docx; hrc memo syria aiding libya 030311.docx March 3, 2011 For: HillaryThxH     Huma Abedin Fw: H: Latest: How Syria aiding Qaddafi and more... Sid hrc memo syria aiding libya 030311.docx Pis print.Pis print. -•-...-^ H     °Russorv@state.gov' Fw: Meet The Right-Wing Extremist Behind Anti-fvluslim Film That Sparked Deadly Riots From [meat) Sent:   To: 11 Subject: Meet The Right Wing Extremist Behind Anti-Muslim Film That Sparked Deadly Riots htte/maxbiumenthal.com12012/09/meet-the-right-wing-extremist-behind-anti-musiim-tihn-that-sparked- deadly-riots/ Sent from Verizon Wireless 4G LTE DROID U.S. Department State Case No. F-2015-04841 Doc No. C05739559 Date: 05/13/2015 STATE DEPT. - PRODUCED HOUSE SELECT BENGHAZI COMM. SUBJECT AGREEMENT SENSITIVE INFORMATION & REDACTIONS. FOIA WAIVER. STATE-5CB0045251H     Huma Abedin Fw: H: Latest: How Syria aiding Qaddafi and more... Sid hrc memo Syria aiding

In [25]:
stopwords = nltk.corpus.stopwords.words('english')
english_vocab = set(w.lower() for w in nltk.corpus.words.words())
noStopWords = [word for word in nltk.word_tokenize(fifthAttempt) if word not in stopwords]
englishWords = [word for word in noStopWords if word in english_vocab]

In [26]:
englishWords[:10]

['memo',
 'memo',
 'memo',
 'print',
 'meat',
 'memo',
 'sept',
 'direct',
 'sept',
 'direct']

In [27]:
englishString = ""
for i in range(len(englishWords)):
    englishString = englishString + " " + englishWords[i]

In [28]:
englishString[:100]

' memo memo memo print meat memo sept direct sept direct piece appear page tomorrow make sure profess'

In [29]:
wordcloud5 = WordCloud().generate(englishString)
# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud5)
plt.axis("off")

# lower max_font_size
wordcloud5 = WordCloud(max_font_size=40).generate(englishString)
plt.figure()
plt.imshow(wordcloud5)
plt.axis("off")
plt.show()

In [45]:
from nltk.corpus import verbnet
verbnet.lemmas()[:25]
noVerbs = [word for word in englishWords if word not in verbnet.lemmas()]

In [47]:
noVerbsString = ""
for i in range(len(noVerbs)):
    noVerbsString = noVerbsString + " " + noVerbs[i]

In [48]:
wordcloud6 = WordCloud().generate(noVerbsString)
# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud6)
plt.axis("off")

# lower max_font_size
wordcloud6 = WordCloud(max_font_size=40).generate(noVerbsString)
plt.figure()
plt.imshow(wordcloud6)
plt.axis("off")
plt.show()

In [30]:
stemmer = SnowballStemmer("english")

In [31]:
import nltk
import codecs
from sklearn import feature_extraction

In [32]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [33]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [34]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [35]:
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [36]:
tokenize_only(extractedBadyText5.to_string())

NameError: name 'extractedBadyText5' is not defined